In [2]:
import pandas as pd

# Work on syllogism form

In [5]:
df_form = pd.read_csv("./data/form_data.csv")

In [18]:
absurde = ["AO3","AO4","OA3","OA4"]
positif = ["AA1","AA2","AA4","AI2","AI4","IA1","IA4"]
universel = ["AA1","AA2","AA4","AE1","AE2","AE3","AE4","EA1","EA2","EA3","EA4"]
particulier = ["AI2","AI4","AO3","AO4","EI1","EI2","EI3","EI4","IA1","IA4","IE1","IE2","IE3","IE4","OA3","OA4"]

In [19]:
df_form_valide = df_form.loc[df_form["has_conclusion"]==True,:]

In [21]:
df_form_valide["mode"] = df_form_valide.task_form.apply(lambda x: "universel" if x in universel else "particulier")
df_form_valide["signe"] = df_form_valide.task_form.apply(lambda x: "positif" if x in positif else "negatif")
df_form_valide["absurde"] = df_form_valide.task_form.apply(lambda x: True if x in absurde else False)

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_76729/1893931228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_form_valide["mode"] = df_form_valide.task_form.apply(lambda x: "universel" if x in universel else "particulier")
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_76729/1893931228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_form_valide["signe"] = df_form_valide.task_form.apply(lambda x: "positif" if x in positif else "negatif")
/var/folders/py/4rt338cj1ks4z33

In [23]:
df_form_valide = df_form_valide.drop("Unnamed: 0", axis=1)

In [27]:
df_form_valide.to_csv("./data/form_data_final.csv", index=False)

# Work on nlp_resul

In [29]:
df_nlp_result = pd.read_csv("./data/nlp_result.csv")
df_idseq = pd.read_csv("./data/idseq_sentence.csv")

In [74]:
df_nlp_result["task_form"].value_counts()

IE4    109
II3    109
EO3    107
EO1    107
AI3    107
      ... 
AO4     96
OA3     95
IA4     94
IO4     93
IA1     93
Name: task_form, Length: 64, dtype: int64

In [42]:
df_nlp_result.shape

(6506, 7)

In [43]:
df_idseq.shape

(6506, 2)

In [35]:
df_nlp_result.head()

,Unnamed: 0,id_seq,task_form,best_bert_00_index,bert_00success,best_mnli_00_index,mnli_00success
0,0,1_0_R,IA4,2,1,2,1
1,1,1_1_R,EA3,0,0,7,1
2,2,1_2_R,AI2,3,1,2,1
3,3,1_3_R,AA1,0,1,4,0
4,4,1_4_R,OA4,4,0,5,1


In [34]:
df_idseq.dtypes

sentenced    object
id_seq       object
dtype: object

In [36]:
df_idseq.head()

,sentenced,id_seq
0,Some models are managers and All models are cl...,1_0_R
1,No divers are carpenters and All linguists are...,1_1_R
2,All therapists are climbers and Some skaters a...,1_2_R
3,All bankers are golfers and All golfers are te...,1_3_R
4,Some boxers are not opticians and All boxers a...,1_4_R


In [38]:
df_nlp_final = df_nlp_result.merge(df_idseq, how="left", on ='id_seq')

In [41]:
df_nlp_final = df_nlp_final.drop(["Unnamed: 0","id_seq"], axis = 1)

In [44]:
df_nlp_final.to_csv("./data/nlp_result_final.csv", index=False)

# Analyse subset

In [ ]:
df_form = pd.read_csv("./data/form_data_final.csv")
df_nlp_result = pd.read_csv("./data/nlp_result_final.csv")

In [77]:
df_sub= df_nlp_result.merge(df_form, how="left", on='task_form')

In [78]:
df_sub.shape

(6506, 12)

In [80]:
df_sub.columns

Index(['task_form', 'best_bert_00_index', 'bert_00success',
       'best_mnli_00_index', 'mnli_00success', 'sentenced', 'has_conclusion',
       'index_conclusion', 'implicatur', 'mode', 'signe', 'absurde'],
      dtype='object')

In [82]:
df_sub.loc[ df_sub['has_conclusion']==True,['bert_00success', 'mnli_00success' ]].shape

(2709, 2)

In [81]:
df_sub.loc[ df_sub['has_conclusion']==True,['bert_00success', 'mnli_00success' ]].mean()

bert_00success    0.194168
mnli_00success    0.494647
dtype: float64

# Final data set

In [3]:
df_nlp_result = pd.read_csv("./data/nlp_result_final.csv")
df_form = pd.read_csv("./data/form_data_final.csv")
df_human = pd.read_csv("./data/human_data.csv")

In [67]:
df_final = df_human.merge(df_nlp_result, how="left", on ='sentenced')
df_final = df_final.merge(df_form, how="left", left_on='task_form_x', right_on = "task_form")

In [68]:
df_final = df_final.dropna()

In [69]:
import ast
df_final["human_success_bool"] = df_final[["human_response_index","index_conclusion"]].apply(lambda x: x[0] in ast.literal_eval(x[1]), axis = 1)
df_final["human_success"] = df_final["human_success_bool"].apply(lambda x: 1 if x == True else 0)

In [70]:
df_final = df_final[["human_response_index","human_success",'best_bert_00_index', 'bert_00success',
       'best_mnli_00_index', 'mnli_00success','task_form','implicatur', 'mode', 'signe', 'absurde']]

In [73]:
df_final.to_csv("./data/df_final.csv", index=False)

In [71]:
df_final.head()

,human_response_index,human_success,best_bert_00_index,bert_00success,best_mnli_00_index,mnli_00success,task_form,implicatur,mode,signe,absurde
0,2,1,2,1,2,1,IA4,False,particulier,positif,False
1,6,1,0,0,7,1,EA3,False,universel,negatif,False
2,3,1,3,1,2,1,AI2,False,particulier,positif,False
3,0,1,0,1,4,0,AA1,False,universel,positif,False
4,5,1,4,0,5,1,OA4,False,particulier,negatif,True


In [72]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4621 entries, 0 to 10952
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   human_response_index  4621 non-null   int64 
 1   human_success         4621 non-null   int64 
 2   best_bert_00_index    4621 non-null   int64 
 3   bert_00success        4621 non-null   int64 
 4   best_mnli_00_index    4621 non-null   int64 
 5   mnli_00success        4621 non-null   int64 
 6   task_form             4621 non-null   object
 7   implicatur            4621 non-null   object
 8   mode                  4621 non-null   object
 9   signe                 4621 non-null   object
 10  absurde               4621 non-null   object
dtypes: int64(6), object(5)
memory usage: 433.2+ KB


# Analyse conclusion type

In [83]:
df_final["best_bert_00_index"].value_counts()

2    1768
0    1332
1     594
3     543
4     188
5     163
7      17
6      16
Name: best_bert_00_index, dtype: int64

In [84]:
df_final["best_mnli_00_index"].value_counts()

6    1182
5    1057
4     759
7     569
0     464
2     416
1      97
3      77
Name: best_mnli_00_index, dtype: int64

In [85]:
df_final["human_response_index"].value_counts()

8    705
6    702
7    611
5    577
4    573
2    543
3    468
0    267
1    175
Name: human_response_index, dtype: int64

In [86]:
df_form = pd.read_csv("./data/form_data_final.csv")


,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
0,IA4,True,"[2, 5]",False,particulier,positif,False
1,EA3,True,"[4, 5, 6, 7]",False,universel,negatif,False
2,AI2,True,"[2, 3]",False,particulier,positif,False
3,AA1,True,"[0, 2, 3]",False,universel,positif,False
4,OA4,True,[5],False,particulier,negatif,True
5,AO3,True,[5],False,particulier,negatif,True
6,AE3,True,"[4, 5, 6, 7]",False,universel,negatif,False
7,EI3,True,[5],False,particulier,negatif,False
8,EI2,True,[5],False,particulier,negatif,False
9,EA1,True,[5],True,universel,negatif,False


In [92]:
import ast

from pandas import DataFrame

my_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict[e] += 1


{0: 1, 1: 1, 2: 7, 3: 5, 4: 12, 5: 14, 6: 4, 7: 4}

In [98]:
df_ccl_freq = pd.Series(my_dict)
df_ccl_freq = df_ccl_freq / df_ccl_freq.sum()
df_ccl_freq

0    0.020833
1    0.020833
2    0.145833
3    0.104167
4    0.250000
5    0.291667
6    0.083333
7    0.083333
dtype: float64

In [100]:
df_ccl_freq.to_csv("./data/ccl_freq.csv")

# When right

In [101]:
df_form.shape

(27, 7)

In [103]:
df_random= pd.Series(my_dict)
df_random = df_random / 27
df_random.mean()

0.2222222222222222

# Random for alls

In [121]:
df_form.columns

Index(['task_form', 'has_conclusion', 'index_conclusion', 'implicatur', 'mode',
       'signe', 'absurde'],
      dtype='object')

In [122]:
df_form["random_succes"] = df_form["index_conclusion"].apply(lambda x : 0.125 * len(ast.literal_eval(x)))

In [129]:
df_form[["random_succes","task_form"]].set_index("task_form").to_csv("./data/random_success.csv")

# Analyse only on negativ

In [ ]:
df_form.signe.value_counts()

negatif    20
positif     7
Name: signe, dtype: int64

In [130]:


df_form_negativ = df_form.loc[df_form.signe == "negatif",:]

df_form_negativ

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde,random_succes
1,EA3,True,"[4, 5, 6, 7]",False,universel,negatif,False,0.500
4,OA4,True,[5],False,particulier,negatif,True,0.125
5,AO3,True,[5],False,particulier,negatif,True,0.125
6,AE3,True,"[4, 5, 6, 7]",False,universel,negatif,False,0.500
7,EI3,True,[5],False,particulier,negatif,False,0.125
8,EI2,True,[5],False,particulier,negatif,False,0.125
9,EA1,True,[5],True,universel,negatif,False,0.125
10,IE3,True,[4],False,particulier,negatif,False,0.125
12,OA3,True,[4],False,particulier,negatif,True,0.125
14,IE4,True,[4],False,particulier,negatif,False,0.125


In [131]:
from pandas import DataFrame
import ast
my_dict_neg = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_negativ.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_neg[e] += 1

In [132]:
df_form_negativ.shape

(20, 8)

In [134]:
my_dict_neg

{0: 0, 1: 0, 2: 0, 3: 0, 4: 12, 5: 12, 6: 4, 7: 4}

In [135]:
df_random_ng= pd.Series(my_dict_neg)
df_random_ng = df_random_ng / 20
df_random_ng

0    0.0
1    0.0
2    0.0
3    0.0
4    0.6
5    0.6
6    0.2
7    0.2
dtype: float64

In [136]:
df_random_ng.mean()

0.2

# Analyse only on positive

In [ ]:


df_form_positive = df_form.loc[df_form.signe == "positif",:]

df_form_positive

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
0,IA4,True,"[2, 5]",False,particulier,positif,False
2,AI2,True,"[2, 3]",False,particulier,positif,False
3,AA1,True,"[0, 2, 3]",False,universel,positif,False
11,AA2,True,"[1, 2, 3]",False,universel,positif,False
13,AI4,True,"[2, 3]",False,particulier,positif,False
15,AA4,True,"[2, 3]",True,universel,positif,False
17,IA1,True,"[2, 5]",False,particulier,positif,False


In [ ]:
from pandas import DataFrame
import ast
my_dict_positiv = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_positive.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_positiv[e] += 1

In [ ]:
df_form_positive.shape

(7, 7)

In [ ]:
df_random_pos= pd.Series(my_dict_positiv)
df_random_pos = df_random_pos / 7
df_random_pos

0    0.142857
1    0.142857
2    1.000000
3    0.714286
4    0.000000
5    0.285714
6    0.000000
7    0.000000
dtype: float64

In [ ]:
df_random_pos.mean()

0.2857142857142857

# Analyse only on implicatur

In [ ]:
df_form.implicatur.value_counts()

False    22
True      5
Name: implicatur, dtype: int64

In [ ]:


df_form_implicatur = df_form.loc[df_form.implicatur == True,:]

df_form_implicatur

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
9,EA1,True,[5],True,universel,negatif,False
15,AA4,True,"[2, 3]",True,universel,positif,False
16,AE4,True,[4],True,universel,negatif,False
23,AE2,True,[4],True,universel,negatif,False
25,EA4,True,[5],True,universel,negatif,False


In [ ]:
from pandas import DataFrame
import ast
my_dict_implicatur = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_implicatur.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_implicatur[e] += 1

In [ ]:
df_random_imp= pd.Series(my_dict_implicatur)
df_random_imp = df_random_imp / 5
df_random_imp

0    0.0
1    0.0
2    0.2
3    0.2
4    0.4
5    0.4
6    0.0
7    0.0
dtype: float64

In [ ]:
df_random_imp.mean()

0.15000000000000002

# Analyse only on not implicatur

In [ ]:
df_form.implicatur.value_counts()

False    22
True      5
Name: implicatur, dtype: int64

In [10]:


df_form_not_implicatur = df_form.loc[df_form.implicatur == False,:]

df_form_not_implicatur

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
0,IA4,True,"[2, 5]",False,particulier,positif,False
1,EA3,True,"[4, 5, 6, 7]",False,universel,negatif,False
2,AI2,True,"[2, 3]",False,particulier,positif,False
3,AA1,True,"[0, 2, 3]",False,universel,positif,False
4,OA4,True,[5],False,particulier,negatif,True
5,AO3,True,[5],False,particulier,negatif,True
6,AE3,True,"[4, 5, 6, 7]",False,universel,negatif,False
7,EI3,True,[5],False,particulier,negatif,False
8,EI2,True,[5],False,particulier,negatif,False
10,IE3,True,[4],False,particulier,negatif,False


In [11]:
from pandas import DataFrame
import ast
my_dict_not_implicatur = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_not_implicatur.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_not_implicatur[e] += 1

In [13]:
df_random_not_imp= pd.Series(my_dict_not_implicatur)
df_random_not_imp = df_random_not_imp / 22
df_random_not_imp

0    0.045455
1    0.045455
2    0.272727
3    0.181818
4    0.454545
5    0.545455
6    0.181818
7    0.181818
dtype: float64

In [14]:
df_random_not_imp.mean()

0.23863636363636365

# Analyse only on absurde

In [15]:
df_form.absurde.value_counts()

False    23
True      4
Name: absurde, dtype: int64

In [16]:


df_form_absurde = df_form.loc[df_form.absurde == True,:]

df_form_absurde

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
4,OA4,True,[5],False,particulier,negatif,True
5,AO3,True,[5],False,particulier,negatif,True
12,OA3,True,[4],False,particulier,negatif,True
20,AO4,True,[4],False,particulier,negatif,True


rep = 12.5

# Analyse only on not absurde

In [17]:
df_form.absurde.value_counts()

False    23
True      4
Name: absurde, dtype: int64

In [18]:


df_form_not_absurde = df_form.loc[df_form.absurde == False,:]

df_form_not_absurde

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
0,IA4,True,"[2, 5]",False,particulier,positif,False
1,EA3,True,"[4, 5, 6, 7]",False,universel,negatif,False
2,AI2,True,"[2, 3]",False,particulier,positif,False
3,AA1,True,"[0, 2, 3]",False,universel,positif,False
6,AE3,True,"[4, 5, 6, 7]",False,universel,negatif,False
7,EI3,True,[5],False,particulier,negatif,False
8,EI2,True,[5],False,particulier,negatif,False
9,EA1,True,[5],True,universel,negatif,False
10,IE3,True,[4],False,particulier,negatif,False
11,AA2,True,"[1, 2, 3]",False,universel,positif,False


In [20]:
from pandas import DataFrame
import ast
my_dict_not_absurde= {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_not_absurde.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_not_absurde[e] += 1

In [21]:
df_random_not_abs= pd.Series(my_dict_not_absurde)
df_random_not_abs = df_random_not_abs / 23
df_random_not_abs

0    0.043478
1    0.043478
2    0.304348
3    0.217391
4    0.434783
5    0.521739
6    0.173913
7    0.173913
dtype: float64

In [24]:
df_random_not_abs.mean()

0.2391304347826087

# Analyse only on universal

In [26]:
df_form.columns

Index(['task_form', 'has_conclusion', 'index_conclusion', 'implicatur', 'mode',
       'signe', 'absurde'],
      dtype='object')

In [28]:
df_form["mode"].value_counts()

particulier    16
universel      11
Name: mode, dtype: int64

In [29]:


df_form_uni = df_form.loc[df_form["mode"] == "universel",:]

df_form_uni

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
1,EA3,True,"[4, 5, 6, 7]",False,universel,negatif,False
3,AA1,True,"[0, 2, 3]",False,universel,positif,False
6,AE3,True,"[4, 5, 6, 7]",False,universel,negatif,False
9,EA1,True,[5],True,universel,negatif,False
11,AA2,True,"[1, 2, 3]",False,universel,positif,False
15,AA4,True,"[2, 3]",True,universel,positif,False
16,AE4,True,[4],True,universel,negatif,False
18,AE1,True,"[4, 5, 6, 7]",False,universel,negatif,False
21,EA2,True,"[4, 5, 6, 7]",False,universel,negatif,False
23,AE2,True,[4],True,universel,negatif,False


In [30]:
from pandas import DataFrame
import ast
my_dict_uni = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_uni.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_uni[e] += 1

In [31]:
df_random_uni= pd.Series(my_dict_uni)
df_random_uni = df_random_uni / 11
df_random_uni

0    0.090909
1    0.090909
2    0.272727
3    0.272727
4    0.545455
5    0.545455
6    0.363636
7    0.363636
dtype: float64

In [34]:
df_random_uni.mean()

0.3181818181818182

# Analyse only on particular

In [ ]:
df_form.columns

Index(['task_form', 'has_conclusion', 'index_conclusion', 'implicatur', 'mode',
       'signe', 'absurde'],
      dtype='object')

In [ ]:
df_form["mode"].value_counts()

particulier    16
universel      11
Name: mode, dtype: int64

In [35]:


df_form_part = df_form.loc[df_form["mode"] == "particulier",:]

df_form_part

,task_form,has_conclusion,index_conclusion,implicatur,mode,signe,absurde
0,IA4,True,"[2, 5]",False,particulier,positif,False
2,AI2,True,"[2, 3]",False,particulier,positif,False
4,OA4,True,[5],False,particulier,negatif,True
5,AO3,True,[5],False,particulier,negatif,True
7,EI3,True,[5],False,particulier,negatif,False
8,EI2,True,[5],False,particulier,negatif,False
10,IE3,True,[4],False,particulier,negatif,False
12,OA3,True,[4],False,particulier,negatif,True
13,AI4,True,"[2, 3]",False,particulier,positif,False
14,IE4,True,[4],False,particulier,negatif,False


In [36]:
from pandas import DataFrame
import ast
my_dict_part = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for row in df_form_part.index_conclusion:
    for e in ast.literal_eval(row):
        my_dict_part[e] += 1

In [37]:
df_random_part= pd.Series(my_dict_part)
df_random_part = df_random_part / 16
df_random_part

0    0.000
1    0.000
2    0.250
3    0.125
4    0.375
5    0.500
6    0.000
7    0.000
dtype: float64

In [38]:
df_random_part.mean()

0.15625